# Rust Crash Course - 03 - Ownership and Borrowing

The central concept of Rust is called *Ownership*. It enables memory safety at compile time without requiring a garbage collector.

In the following, ownership, borrowing, and related structures are explained.

The contents represent a brief and compact introduction to the topic, inspired by the [Rust Book](https://doc.rust-lang.org/book/), the [Rust Reference](https://doc.rust-lang.org/reference/), and [Rust By Example](https://doc.rust-lang.org/rust-by-example/).

## Heap-Allocated Data

Ownership is important for scalar types, but becomes even more important for data types that allocate memory on the heap. Therefore, in the subsequent cells, new heap-based data types are introduced. Such data types are also called *smart pointers*, because they point at data on the heap and usually carry some additional meta information.

### ``Box<T>``

The ``Box`` type allocates memory on the heap and places a variable into it.

The value can be accessed by dereferencing with a preceding ``*``.

For details see https://doc.rust-lang.org/std/boxed/struct.Box.html.

In [ ]:
let ints: [u64; 8] = [1, 2, 3, 4, 5, 6, 7, 8];
let heap_ints = Box::new(ints);
let stack_ints = *heap_ints;    // copy of heap_ints on the stack

println!("ints               = {:?} and uses {} byte(s) on the stack!",
    ints, std::mem::size_of_val(&ints));
println!("heap_ints          = {:?} and uses {} byte(s) on the stack!",
    heap_ints, std::mem::size_of_val(&heap_ints));
println!("stack_ints         = {:?} and uses {} byte(s) on the stack!",
    stack_ints, std::mem::size_of_val(&stack_ints));

let heap_ints_oneliner: Box<[u64; 8]> = Box::new([9, 8, 7, 6, 5, 4, 3, 2]);

println!("heap_ints_oneliner = {:?} and uses {} byte(s) on the stack!",
    heap_ints_oneliner, std::mem::size_of_val(&heap_ints_oneliner));

### ``Vec<T>``

The vector type ``Vec`` is a growable array type that allocates memory on the heap.

For details see https://doc.rust-lang.org/std/vec/struct.Vec.html.

In [ ]:
let mut vector_for_floats: Vec<f64> = Vec::new();
vector_for_floats.push(3.17);
vector_for_floats.push(3.23);
vector_for_floats.push(3.76);
vector_for_floats.push(3.51);
vector_for_floats.push(3.32);

for f in &vector_for_floats {            // "for .. in .." enables iterating over vector
    println!("{}", f);
};

println!("vector_for_floats uses {} byte(s) on the stack!", std::mem::size_of_val(&vector_for_floats));

The macro ``vec!`` can be used to initialize vectors in a short, convenient way.

In [ ]:
let v_int = vec![0, 1, 2, 3];

println!("v_int[2] = {}", v_int[2]);

println!("v_int uses {} byte(s) on the stack!", std::mem::size_of_val(&v_int));

### ``String``

A ``String`` variable consists of two parts:
- on the heap: raw string data (dynamically allocated as needed)
- on the stack: pointer to string data on heap, length of current string, whole capacity of string

On a byte level, a ``String`` variable is UTF-8 encoded, i.e. individual characters may be represented by 1 to 4 bytes.

The type ``str`` is closely related to ``String``. It is called a 'string slice' and is usually used in its borrowed form, ``&str``.

For details see https://doc.rust-lang.org/std/string/struct.String.html and https://doc.rust-lang.org/std/primitive.str.html.

In [ ]:
let mut string_var = String::from("This is");
string_var.push(' ');           // append single character
let ok_str = "OK";              // implicit type: &'static str
string_var.push_str(ok_str);    // append string slice
string_var.push_str("! ");      // append string literal
string_var.push('👍');

let string_str = string_var.clone();

println!("string_var          = \'{}\' and uses {} byte(s) on the stack! (further, the raw data is on the heap)",
    string_var, std::mem::size_of_val(&string_var));
println!("string_str.as_str() = \'{}\' and uses {} byte(s) on the stack! (because it is a static string slice)",
    string_str.as_str(), std::mem::size_of_val(string_str.as_str()));

## Ownership

Ownership manages memory and other resources while maintaining certain guarantees.

There are three important rules:
- each value in Rust has a variable that’s called its owner
- there can only be one owner at a time
- when the owner goes out of scope, the value will be dropped

There are some common cases that are relevant for ownership transfers:
- assignments
- function arguments and return values

The following so-called traits handle ownership transfer in Rust:
- ``Move``: stack data is copied, heap data is not; previous reference on heap data becomes invalid
- ``Clone``: a ''deep''/full copy of stack and heap data is created
- ``Copy``: similar to cloning, but only valid for scalar data types and simple groups thereof

### Assignments

When values of scalar data types or simple groups thereof are assigned, the ``Copy`` trait is called.

In [ ]:
let mut x = 1;
let mut y = x;              // value of scalar data type is copied

println!("x = {}", x);      // works as intended
println!("y = {}", y);      // works as intended

However, if one assigns a more complex data type like ``String`` to another variable, the behavior is different.

The following example shows the assignment of ``s1`` to ``s2``. After that operation, ``s2`` owns the corresponding string and ``s1`` looses ownership, i.e. becomes invalid, because the ``Move`` trait is called.

In [ ]:
let s1 = String::from("Try this!");
let s2 = s1;

println!("s1 = {}", s1);              // leads to compiler error, because s1 is not valid anymore
println!("s2 = {}", s2);

If one explicitly aims at performing a deep copy of a ``String`` variable, one has to call the ``clone()`` method.

In [ ]:
let s1 = String::from("Try this!");
let s2 = s1.clone();                  // deep copy is created

println!("s1 = {}", s1);              // works as intended
println!("s2 = {}", s2);              // works as intended

### Scope

Everytime a variable goes out of scope, Rust automatically calls the ``drop()`` function that releases ownership and returns the allocated memory to the operating system. One way to define a scope is by ``{`` and ``}``.

In [ ]:
let mut x = 1;

let y = {                   // new scope
    
    let mut n = 127;        // n is valid from here
    
    n = (n + x) * 2;        // use n and x
    
    n                       // return value of n (and transfer ownership to y)
    
};                          // scope is over, n is not valid any longer

println!("n = {}", n);      // leads to compiler error, because n is not valid anymore

{                           // new scope

    let mut z = 42;         // z is valid from here
    
    z = z + 5;              // use z ...
    
    x = 17;                 // x can be used, too
    
};                          // semicolons in Rust suppress return value of expressions

println!("x = {}", x);      // works as intended, because x is in scope
println!("y = {}", y);      // works as intended, because y got ownership of n
println!("z = {}", z);      // leads to compiler error, because z is not valid anymore

This is also true for more complex data types, like ``String``.

In [ ]:
{

    let mut my_str = String::from("Good");  // the String data type might grow --> heap memory necessary

    my_str.push_str(" Morning!");           // push_str() appends a literal to a String variable

    println!("{}", my_str);                 // prints my_str

};                                          // my_str is dropped

println!("Again: {}", my_str);              // leads to compiler error, because my_str is not valid anymore

In some cases, one might want to return memory even before the end of scope. That can be achieved by explicitly calling ``drop()``.

In [ ]:
let mut my_next_str = String::from("Good");
my_next_str.push_str(" Evening!");

println!("{}", my_next_str);

drop(my_next_str);                           // variable my_next_str is dropped at this point

println!("Again: {}", my_next_str);          // leads to compiler error, because my_next_str is not valid anymore

### Function Arguments and Return Values

The following example illustrates that a variable of type ``String`` that has been moved into the scope of a function is dropped after leaving the function's scope.

In [ ]:
fn get_string_and_drop(a_str: String) {
    println!("{} Night!", a_str);
}                                        // ownership of a_str is dropped at this point

let mut xyz_str = String::from("Good");

get_string_and_drop(xyz_str);            // xyz_str is moved to function and dropped after processing

println!("xyz_str = {}", xyz_str);       // leads to compiler error, because xyz_str is not valid anymore

On the other hand, returning a value from a function will transfer ownership to the calling scope.

In [ ]:
fn give_abc() -> String {             
    let abc_str = String::from("abc");
    abc_str                                   // value of abc_str and ownership is returned to caller
}

let received_abc = give_abc();                // received_abc takes ownership

println!("Return Value: {}", received_abc);

## Borrowing

Moving ownership in and back out of functions can be cumbersome. Hence, Rust implements a concept called *Borrowing*: Ownership can be temporarily borrowed from a value's owner by using references.

There are two things to consider when working with references:
- At any given time, you can have either *one mutable reference* or *any number of immutable references*.
- References must always be valid.

### Immutable References

The syntax ``&`` creates immutable references.

In [ ]:
fn get_len(s: &String) -> usize {
    s.len()
}

let str_1 = String::from("Wonderful Day!");

let len = get_len(&str_1);                            // immutable reference to str_1

println!("The length of '{}' is {}.", str_1, len);

### Mutable References

Mutable references can be created by using ``&mut``.

In [ ]:
fn append_year(s: &mut String) {
    s.push_str("2020");
}

let mut str_2 = String::from("Wonderful Year: ");

append_year(&mut str_2);                             // mutable reference to str_2

println!("New String: \'{}\'", str_2);

### Dangling References (Pointers)

In contrast to some other languages, e.g. C, the Rust compiler will not let you create dangling, invalid references.

In [ ]:
fn get_dangling_ref() -> String {
    let s = String::from("Test String!");
    &s
}                                            // s is dropped here

let dangling_ref = get_dangling_ref();       // leads to compiler error, because &s is not valid anymore

println!("String behind dangling reference: {}", dangling_ref);

### Slices

Slices are immutable references to a part of a collection.

Note the range syntax ``<start-element-inclusive>..<end-element-exclusive>``, where one or both limits can be omitted.

This can be especially usefull when accessing parts of a ``String`` variable, for example.

In [ ]:
fn print_alphabet_parts() {
    
    let alphabet = String::from("ABCDEFGHIJKLMNOPQRSTUVWXYZ");

    let abc_str = &alphabet[..3];
    let def_str = &alphabet[3..6];
    let xyz_str = &alphabet[23..];

    println!("abc_str = {}", abc_str);
    println!("def_str = {}", def_str);
    println!("xyz_str = {}", xyz_str);
}

print_alphabet_parts();

The same is possible for arrays.

In [ ]:
fn return_end_of_array(a: &[i32]) -> &[i32] {
    &a[a.len()-3..]
}
    
let a: [i32; 8] = [1, 2, 3, 4, 5, 6, 7, 8];

println!("End of Array is: {:?}", return_end_of_array(&a));

## Exercises

The following exercises may help to practice the obtained knowledge.

### String Comparison

Write a function to count the occurrences of a given string in another given string.

In [ ]:
fn count_str_in_str ( // TODO ) {

    // TODO

}

let long_str = String::from("abhdzweduihb3322baawueounm17oonoduabduauuuuwe23edun12");
let search_str = String::from("ab");
//let search_str = String::from("du");

let search_count = count_str_in_str( // TODO );

println!("Pattern \'{}\' found {} times!", search_str, search_count);

### Vector Manipulation

Write two functions:

- Function 1 takes a vector of integers, calculates the doubled value of the last element, and appends it to this vector.
- Function 2 takes a vector of integers, checks if the vector length is greater than 7, and, if it is, removes the first 3 elements of the vector. Further, it returns ``true`` or ``false``, if the vector was cut or not, respectively.

In [ ]:
fn double_and_append ( // TODO ) {
    
    // TODO
    
}

fn check_and_remove ( // TODO ) {
    
    // TODO

}

// initialization of vector
let mut int_vec: Vec<i32> = [1, 2, 4].to_vec();
println!("--- INITIALIZATION ---");
println!("Vector State: {:?}", int_vec);

// loop through several iterations
for i in 0..12 {
    
    // function calls
    double_and_append( // TODO );
    let shrink_bool = check_and_remove( // TODO );
    
    // print current state
    println!("--- ITERATION {} ---", i);
    if shrink_bool == true {
        println!("Vector Shrinked!");
    }
    println!("Vector State: {:?}", int_vec);
};

println!("--- THE END ---");
println!("Final Vector State: {:?}", int_vec);     // should be [64, 512, 2048, 4096, 8192, 16384]